O jeito mais simples de evitar o overfiting é reduzindo o tamanho do modelo, ou seja reduzindo o número de 
parâmetros (isso é determinado pela quantidade de camadas e neurônios em cada uma delas).

Em Deep Learning, isso é chamado de capacidade de modelo.

Mais parâmetros significa maior capacidade de memorização e mais facilidade em mapear amostras de treino e 
seus targets, contudo, perder poder de generalização, causando Overfiting.

Poucos parâmetros também são um problema, aumentando os erros das previsões, tornando o modelo sem utilidade.

Vamos investigar como o aumento do número de nós(neurônios) pode impactar a capacidade de um modelo de redes neurais; Usaremos o sklearn.datasets.make_blobs para criar um dataste fake. Esse dataset terá 1000 amostras, com 100 features cada.

In [19]:
#prepare multiclass classification dataset
def create_dataset():
    #Gerar um dataset 2d para classificação
    X, y = make_blobs(n_samples=1000, centers=20, n_features=100, cluster_std=2,
                     random_state=2)
    #one hot encode output
    y = to_categorical(y)
    #Dividir o dataset em treino e teste
    n_train = 500
    trainX, testX = X[:n_train, :], X[n_train:, :]
    trainy, testy = y[:n_train, :], y[n_train:]
    return trainX, trainy, testX, testy

In [20]:
#Treina o modelo com um dado número de nós, retorna a acuracy para o conjunto de teste

def evaluate_model(n_nodes, trainX, trainy, testX, testy):
    #configura o modelo baseado nos dados
    n_input, n_classes = trainX.shape[1], testy.shape[1]
    #define a arquitetura do modelo
    modelo = Sequential()
    model.add(Dense(n_nodes, input_dim=n_input,activation='relu',
                    kernel_initializer='he_uniform'))
    model.add(Dense(n_classes, activation='softmax'))
    #compila o modelo
    op = SGD(lr=0.01,momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    #treina o modelo
    history = model.fit(trainX, trainy, epochs=100, verbose=0)
    #avalia o modelo com dados de teste
    _, test_acc = model.evaluate(testX, testy, verbose=0)
    return history, test_acc

Criamos uma função para avaliar o modelo, utilizamos uma rede neural com somente uma camada Densa

In [21]:
#Avalia o modelo e plota a curva de aprendizagem
num_nodes = [1, 2, 3, 4, 8, 16, 32, 64]
for n_nodes in num_nodes:
    #Avalia o modelo dado o número de nós
    history, result = evaluate_model(n_nodes, trainX, trainy, testX, testy)
    #Sumariza a accuracy do teste
    print('nodes=%d: %.3f' % (n_nodes, result))
    #plota a curva de aprendizagem
    pyplot.plot(history.history['loss'], label=str(n_nodes))
#Mostra o plot
pyplot.legend()
pyplot.show()

NameError: name 'trainX' is not defined

O aumento do número de nós ajuda a reduzir o erro e aumentar a accuracy. Mas observe que acima de 16 nós temos overfiting.

Realizamos o mesmo teste com o dataset MNIST fashion, para detecção de obejtos.

In [23]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images / 255.0
test_iamges = test_images / 255.0

def evaluate_model(n_nodes, trainX, trainy, testX, testy):
    n_input, n_classes = trainX.shape[0], testy,shape[0]
    model = Sequential()
    model = Sequential([tf.keras.layers.Flatten(),
                        Dense(n_nodes, activation = tf.nn.relu),
                        Dense(10, activation=tf.nn.softmax)])
    model.compile(optimizer = tf.optimizers.Adam(), loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(training_images, training_labels, epochs=5, verbose=0)
    _, test_acc = model.evaluate(test_iamges, test_labels)
    return history, test_acc
                  
                  

4423680/4422102 [==============================] - 3s 1us/step


Testamos com somente 5 épocas e com um número maior de nós.

In [24]:
num_nodes = [16, 32, 64, 128, 256, 512, 1024, 2048]

for n_nodes in num_nodes:
    history, result = evaluate_model(n_nodes, training_images, 
                                    training_labels, test_images, test_labels)
    print('nodes=%d: %.3f' % (n_nodes, result))
    pyplot.plot(history.history['loss'], label=str(n_nodes))
pyplot.legend()
pyplot.show()

NameError: name 'shape' is not defined

Observamos que há ganhos até o número de 256 neurônios. Após isso o resultado não melhora mais. Com essa análise podemos considerar que não há vantagem em aumentar o número de nós para mais de 256. Deixaria o modelo mais lento para treinar e causaria overfitting.